In [1]:
from importlib import reload
import pretty_midi
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

In [107]:
try:
    reload(midiMethods)
    from modules.midiMethods import *
except:
    import modules.midiMethods as midiMethods
    from modules.midiMethods import *
try:
    reload(dataMethods)
    from modules.dataMethods import *
except:
    import modules.dataMethods as dataMethods
    from modules.dataMethods import *

In [67]:
# test out my function for going to note_bin representation
pm = pretty_midi.PrettyMIDI('midi/basic_test.mid')
noteB2 = pm2note_bin(pm)
print(noteB2)
pm2 = note_bin2pm(noteB2)
pm2.write('midi/basic_test_tested.mid')

[[27, 0.0, 0.0, 2.0, 17.0, 27], [0, 0.0, 0.0, 2.0, 19.0, 28], [5, 2.0, 15.0, 2.0, 9.0, 25], [32, 0.0, 0.0, 2.0, 13.0, 26], [2, 2.0, 7.0, 0.0, 42.0, 17], [34, 0.0, 0.0, 0.0, 87.0, 19], [3, 0.0, 42.0, 0.0, 40.0, 11], [5, 0.0, 38.0, 0.0, 42.0, 11], [32, 0.0, 3.0, 1.0, 0.0, 13], [2, 0.0, 42.0, 0.0, 57.0, 9], [7, 0.0, 59.0, 1.0, 52.0, 8], [31, 0.0, 2.0, 1.0, 49.0, 5]]


2.17
2.19
2.09
2.13
0.42
0.87
0.4
0.42
1.0
0.57
1.52
1.49


array([0.        , 0.01127142, 0.02253909, ..., 0.        , 0.        ,
       0.        ])